In [1]:
import pandas as pd
from table_insert_querries import *
from database_operations import create_database_connection, create_tables

In [2]:
%run database_operations.py

In [3]:
curr, conn = create_database_connection()

In [4]:
create_tables(curr)

In [5]:
def insert_records_in_table(records, querry):
    for record in records:
        curr.execute(querry, record)
    print("Records inserted into the table")

### Creating dataframes by reading from subscription csv files

In [6]:
df_subscription_created = pd.read_csv("data/subscription_created.csv") 

In [7]:
df_subscription_created['event_name'] = "subscription_created"

In [8]:
df_subscription_payment_completed = pd.read_csv("data/subscription_payment_completed.csv")

In [9]:
df_subscription_payment_completed['event_name'] = 'subscription_payment_completed'

In [10]:
df_subscription_canceled = pd.read_csv("data/subscription_canceled.csv")

In [11]:
df_subscription_canceled['event_name'] = 'subscription_canceled'

In [12]:
df_subscription_reactivated = pd.read_csv("data/subscription_reactivated.csv")

In [13]:
df_subscription_reactivated['event_name'] = 'subscription_reactivated'

### Combining the datafames from all subscription events into one dataframe

In [14]:
combined_events_df = pd.concat([df_subscription_created, 
                                df_subscription_payment_completed,
                                df_subscription_canceled,
                                df_subscription_reactivated],
                               ignore_index=True)

In [15]:
combined_events_df.head(6)

,id,received_at,original_timestamp,account_id,organization_id,currency,account_sites_count,billing_cycle,sites_2000_daily_pageviews,sites_20000_daily_pageviews,sites_200000_daily_pageviews,sites_200000_plus_daily_pageviews,url_event_was_triggered_on,event_name
0,acd43f22-d309-11e9-a770-00e04c6803e4,2019-08-27 14:17:15.918335,2019-08-27 14:17:11.918335,acc-13442,['org-49772'],eur,12,monthly,2,3,4,3,http://www.gdgt1cpt2ri0.com/w7dsvsbk/7n0ztux3/...,subscription_created
1,acd43f23-d309-11e9-a770-00e04c6803e4,2019-08-27 14:17:15.918335,2019-08-27 14:17:11.918335,acc-13025,['org-46185'],eur,13,yearly,1,4,5,3,http://www.qveak79goxia.com/8m6x73kjctp7m/yiiz...,subscription_created
2,acd43f24-d309-11e9-a770-00e04c6803e4,2019-08-27 14:17:15.918335,2019-08-27 14:17:14.918335,acc-11689,"['org-48692', 'org-35895']",eur,17,monthly,5,6,5,1,http://www.db4szx9qcg1aj.com/,subscription_created
3,acd43f25-d309-11e9-a770-00e04c6803e4,2019-08-27 14:17:15.918335,2019-08-27 14:17:13.918335,acc-16681,"['org-45640', 'org-31339', 'org-35178']",usd,5,yearly,3,1,1,0,http://www.l2mdwl4jor0aid79.com/m8lcy9g2qnnk/1...,subscription_created
4,acd43f26-d309-11e9-a770-00e04c6803e4,2019-08-27 14:17:15.918335,2019-08-27 14:17:12.918335,acc-16206,"['org-39940', 'org-37829', 'org-32434']",usd,13,yearly,10,1,1,1,http://www.dxypuiklsc98e.com/58c9/p2061jvbu0/u...,subscription_created
5,acd43f27-d309-11e9-a770-00e04c6803e4,2019-08-27 14:17:15.918335,2019-08-27 14:17:15.918335,acc-19671,['org-34213'],eur,17,yearly,6,6,2,3,http://www.be2drnkz2o4yawg.com/,subscription_created


### Page views dimension information

In [16]:
page_views_dim_df = combined_events_df[['id', 'sites_2000_daily_pageviews', 
                                        'sites_20000_daily_pageviews', 
                                        'sites_200000_plus_daily_pageviews']]

In [17]:
page_views_dim_df.head(6)

,id,sites_2000_daily_pageviews,sites_20000_daily_pageviews,sites_200000_plus_daily_pageviews
0,acd43f22-d309-11e9-a770-00e04c6803e4,2,3,3
1,acd43f23-d309-11e9-a770-00e04c6803e4,1,4,3
2,acd43f24-d309-11e9-a770-00e04c6803e4,5,6,1
3,acd43f25-d309-11e9-a770-00e04c6803e4,3,1,0
4,acd43f26-d309-11e9-a770-00e04c6803e4,10,1,1
5,acd43f27-d309-11e9-a770-00e04c6803e4,6,6,3


In [18]:
page_views_dim_df_melted = pd.melt(page_views_dim_df, id_vars=["id"])

In [19]:
page_views_dim__name_df = page_views_dim_df_melted[['variable']].drop_duplicates()

In [20]:
page_views_dim__name_df

,variable
0,sites_2000_daily_pageviews
250000,sites_20000_daily_pageviews
500000,sites_200000_plus_daily_pageviews


In [21]:
page_views_dim__name_df_records = page_views_dim__name_df.values.tolist()

In [22]:
page_views_dim__name_df_records

[['sites_2000_daily_pageviews'],
 ['sites_20000_daily_pageviews'],
 ['sites_200000_plus_daily_pageviews']]

### Inserting page views dimension records into  page_views_dim table

In [23]:
insert_records_in_table(page_views_dim__name_df_records,page_view_dim_table_insert)

Records inserted into the table


### Page View Frequency Fact Information

In [24]:
page_view_dim_info = pd.read_sql('select * from page_views_dim', con=conn)

In [25]:
page_view_dim_info

,id,name
0,1,sites_2000_daily_pageviews
1,2,sites_20000_daily_pageviews
2,3,sites_200000_plus_daily_pageviews


In [26]:
def fill_page_view_id(row):
    if row['variable'] == 'sites_2000_daily_pageviews':
        return 1
    elif row['variable'] == 'sites_20000_daily_pageviews':
        return 2
    else:
        return 3

In [27]:
page_views_dim_df_melted['page_view_id'] = page_views_dim_df_melted.apply(fill_page_view_id,
                                                                         axis=1)

In [28]:
page_view_frequency_df = page_views_dim_df_melted[['id', 'page_view_id', 'value']]

In [29]:
page_view_frequency_df.head(4)

,id,page_view_id,value
0,acd43f22-d309-11e9-a770-00e04c6803e4,1,2
1,acd43f23-d309-11e9-a770-00e04c6803e4,1,1
2,acd43f24-d309-11e9-a770-00e04c6803e4,1,5
3,acd43f25-d309-11e9-a770-00e04c6803e4,1,3


In [30]:
page_view_frequency_records = page_view_frequency_df.values.tolist()

In [31]:
page_view_frequency_records[0:3]

[['acd43f22-d309-11e9-a770-00e04c6803e4', 1, 2],
 ['acd43f23-d309-11e9-a770-00e04c6803e4', 1, 1],
 ['acd43f24-d309-11e9-a770-00e04c6803e4', 1, 5]]

### insert page view frequency records into page_view_frequency_fact table

In [32]:
insert_records_in_table(page_view_frequency_records,page_view_frequency_fact_table_insert)

Records inserted into the table


### Currency dimension  information

In [33]:
currency_dim_df = combined_events_df[['currency']].drop_duplicates()

In [34]:
currency_dim_df 

,currency
0,eur
3,usd


In [35]:
currency_dim_df_records = currency_dim_df.values.tolist()

In [36]:
currency_dim_df_records

[['eur'], ['usd']]

### Inserting currency dimension records into currency_dim table

In [37]:
insert_records_in_table(currency_dim_df_records, currency_dim_table_insert)

Records inserted into the table


### Billing cycle dimension information 

In [38]:
billing_cycle_dim_df = combined_events_df[['billing_cycle']].drop_duplicates()

In [39]:
billing_cycle_dim_df.count()

billing_cycle    2
dtype: int64

In [40]:
billing_cycle_records = billing_cycle_dim_df.values.tolist()

In [41]:
billing_cycle_records

[['monthly'], ['yearly']]

### inserting billing cycle dimension records into billing_cycle_dim table

In [42]:
insert_records_in_table(billing_cycle_records, billing_cycle_dim_table_insert)

Records inserted into the table


### Event information (dimension)

In [43]:
event_dim_df = combined_events_df [['id', 'event_name', 'original_timestamp', 
                    'received_at','url_event_was_triggered_on']]

In [44]:
event_dim_df.head(5)

,id,event_name,original_timestamp,received_at,url_event_was_triggered_on
0,acd43f22-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:11.918335,2019-08-27 14:17:15.918335,http://www.gdgt1cpt2ri0.com/w7dsvsbk/7n0ztux3/...
1,acd43f23-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:11.918335,2019-08-27 14:17:15.918335,http://www.qveak79goxia.com/8m6x73kjctp7m/yiiz...
2,acd43f24-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:14.918335,2019-08-27 14:17:15.918335,http://www.db4szx9qcg1aj.com/
3,acd43f25-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:13.918335,2019-08-27 14:17:15.918335,http://www.l2mdwl4jor0aid79.com/m8lcy9g2qnnk/1...
4,acd43f26-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:12.918335,2019-08-27 14:17:15.918335,http://www.dxypuiklsc98e.com/58c9/p2061jvbu0/u...


In [45]:
event_dim_records = event_dim_df.values.tolist()

In [46]:
event_dim_records[0:2]

[['acd43f22-d309-11e9-a770-00e04c6803e4',
  'subscription_created',
  '2019-08-27 14:17:11.918335',
  '2019-08-27 14:17:15.918335',
  'http://www.gdgt1cpt2ri0.com/w7dsvsbk/7n0ztux3/na8q1hclegckvlo/zxilje/lkle2pfl9q/6unie2ojck/wmxq3qn1ec'],
 ['acd43f23-d309-11e9-a770-00e04c6803e4',
  'subscription_created',
  '2019-08-27 14:17:11.918335',
  '2019-08-27 14:17:15.918335',
  'http://www.qveak79goxia.com/8m6x73kjctp7m/yiizx0vab9/ioxjnf/puiqs8qngufe/i8ebttp7w/3aoet31o6n7kbzuy']]

### inserting event dimension records into events_information_dim table

In [47]:
insert_records_in_table(event_dim_records, event_info_dim_table_insert)

Records inserted into the table


### Account dimension information

In [48]:
account_dim_df = combined_events_df[['id','account_id', 'organization_id']]

In [49]:
account_dim_df.head(5)

,id,account_id,organization_id
0,acd43f22-d309-11e9-a770-00e04c6803e4,acc-13442,['org-49772']
1,acd43f23-d309-11e9-a770-00e04c6803e4,acc-13025,['org-46185']
2,acd43f24-d309-11e9-a770-00e04c6803e4,acc-11689,"['org-48692', 'org-35895']"
3,acd43f25-d309-11e9-a770-00e04c6803e4,acc-16681,"['org-45640', 'org-31339', 'org-35178']"
4,acd43f26-d309-11e9-a770-00e04c6803e4,acc-16206,"['org-39940', 'org-37829', 'org-32434']"


In [50]:
account_dim_df_records = account_dim_df.values.tolist()

In [51]:
account_dim_df_records[0:2]

[['acd43f22-d309-11e9-a770-00e04c6803e4', 'acc-13442', "['org-49772']"],
 ['acd43f23-d309-11e9-a770-00e04c6803e4', 'acc-13025', "['org-46185']"]]

### Inserting account dimension records into accounts_information_dim table

In [52]:
insert_records_in_table(account_dim_df_records, account_informartion_dim_table_insert)

Records inserted into the table


#### Events information (facts)

In [53]:
events_fact_df = combined_events_df[['id','account_sites_count']]

In [54]:
events_fact_df.head(5)

,id,account_sites_count
0,acd43f22-d309-11e9-a770-00e04c6803e4,12
1,acd43f23-d309-11e9-a770-00e04c6803e4,13
2,acd43f24-d309-11e9-a770-00e04c6803e4,17
3,acd43f25-d309-11e9-a770-00e04c6803e4,5
4,acd43f26-d309-11e9-a770-00e04c6803e4,13


In [55]:
events_information_dim_df = pd.read_sql('select * from events_information_dim', con=conn)

In [56]:
events_information_dim_df.head(5)

,id,event_id,event_name,original_timestamp,recieved_at,url_event_was_triggered_on
0,1,acd43f22-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:11.918335,2019-08-27 14:17:15.918335,http://www.gdgt1cpt2ri0.com/w7dsvsbk/7n0ztux3/...
1,2,acd43f23-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:11.918335,2019-08-27 14:17:15.918335,http://www.qveak79goxia.com/8m6x73kjctp7m/yiiz...
2,3,acd43f24-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:14.918335,2019-08-27 14:17:15.918335,http://www.db4szx9qcg1aj.com/
3,4,acd43f25-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:13.918335,2019-08-27 14:17:15.918335,http://www.l2mdwl4jor0aid79.com/m8lcy9g2qnnk/1...
4,5,acd43f26-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:12.918335,2019-08-27 14:17:15.918335,http://www.dxypuiklsc98e.com/58c9/p2061jvbu0/u...


In [57]:
events_information_dim_df = events_information_dim_df.rename(columns={'id': 'event_information_id',
                        'event_id': 'id'})

In [58]:
events_information_dim_df.head(5)

,event_information_id,id,event_name,original_timestamp,recieved_at,url_event_was_triggered_on
0,1,acd43f22-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:11.918335,2019-08-27 14:17:15.918335,http://www.gdgt1cpt2ri0.com/w7dsvsbk/7n0ztux3/...
1,2,acd43f23-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:11.918335,2019-08-27 14:17:15.918335,http://www.qveak79goxia.com/8m6x73kjctp7m/yiiz...
2,3,acd43f24-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:14.918335,2019-08-27 14:17:15.918335,http://www.db4szx9qcg1aj.com/
3,4,acd43f25-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:13.918335,2019-08-27 14:17:15.918335,http://www.l2mdwl4jor0aid79.com/m8lcy9g2qnnk/1...
4,5,acd43f26-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:12.918335,2019-08-27 14:17:15.918335,http://www.dxypuiklsc98e.com/58c9/p2061jvbu0/u...


In [59]:
events_merge_df = pd.merge(events_information_dim_df, events_fact_df,  on="id")

In [60]:
events_merge_df.head(5)

,event_information_id,id,event_name,original_timestamp,recieved_at,url_event_was_triggered_on,account_sites_count
0,1,acd43f22-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:11.918335,2019-08-27 14:17:15.918335,http://www.gdgt1cpt2ri0.com/w7dsvsbk/7n0ztux3/...,12
1,2,acd43f23-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:11.918335,2019-08-27 14:17:15.918335,http://www.qveak79goxia.com/8m6x73kjctp7m/yiiz...,13
2,3,acd43f24-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:14.918335,2019-08-27 14:17:15.918335,http://www.db4szx9qcg1aj.com/,17
3,4,acd43f25-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:13.918335,2019-08-27 14:17:15.918335,http://www.l2mdwl4jor0aid79.com/m8lcy9g2qnnk/1...,5
4,5,acd43f26-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:12.918335,2019-08-27 14:17:15.918335,http://www.dxypuiklsc98e.com/58c9/p2061jvbu0/u...,13


In [61]:
accounts_dim_info_df = pd.read_sql('select * from accounts_information_dim', con=conn)

In [62]:
accounts_dim_info_df.head(5)

,id,event_id,account_id,organization_id
0,1,acd43f22-d309-11e9-a770-00e04c6803e4,acc-13442,['org-49772']
1,2,acd43f23-d309-11e9-a770-00e04c6803e4,acc-13025,['org-46185']
2,3,acd43f24-d309-11e9-a770-00e04c6803e4,acc-11689,"['org-48692', 'org-35895']"
3,4,acd43f25-d309-11e9-a770-00e04c6803e4,acc-16681,"['org-45640', 'org-31339', 'org-35178']"
4,5,acd43f26-d309-11e9-a770-00e04c6803e4,acc-16206,"['org-39940', 'org-37829', 'org-32434']"


In [63]:
accounts_dim_info_df = accounts_dim_info_df.rename(columns={'id': 'account_information_id',
                                                           'event_id': 'id'})

In [64]:
accounts_dim_info_df.head(5)

,account_information_id,id,account_id,organization_id
0,1,acd43f22-d309-11e9-a770-00e04c6803e4,acc-13442,['org-49772']
1,2,acd43f23-d309-11e9-a770-00e04c6803e4,acc-13025,['org-46185']
2,3,acd43f24-d309-11e9-a770-00e04c6803e4,acc-11689,"['org-48692', 'org-35895']"
3,4,acd43f25-d309-11e9-a770-00e04c6803e4,acc-16681,"['org-45640', 'org-31339', 'org-35178']"
4,5,acd43f26-d309-11e9-a770-00e04c6803e4,acc-16206,"['org-39940', 'org-37829', 'org-32434']"


In [65]:
events_account_merge_df = pd.merge(events_merge_df,
                                   accounts_dim_info_df,
                                  on='id')

In [66]:
events_account_merge_df.head(5)

,event_information_id,id,event_name,original_timestamp,recieved_at,url_event_was_triggered_on,account_sites_count,account_information_id,account_id,organization_id
0,1,acd43f22-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:11.918335,2019-08-27 14:17:15.918335,http://www.gdgt1cpt2ri0.com/w7dsvsbk/7n0ztux3/...,12,1,acc-13442,['org-49772']
1,2,acd43f23-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:11.918335,2019-08-27 14:17:15.918335,http://www.qveak79goxia.com/8m6x73kjctp7m/yiiz...,13,2,acc-13025,['org-46185']
2,3,acd43f24-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:14.918335,2019-08-27 14:17:15.918335,http://www.db4szx9qcg1aj.com/,17,3,acc-11689,"['org-48692', 'org-35895']"
3,4,acd43f25-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:13.918335,2019-08-27 14:17:15.918335,http://www.l2mdwl4jor0aid79.com/m8lcy9g2qnnk/1...,5,4,acc-16681,"['org-45640', 'org-31339', 'org-35178']"
4,5,acd43f26-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:12.918335,2019-08-27 14:17:15.918335,http://www.dxypuiklsc98e.com/58c9/p2061jvbu0/u...,13,5,acc-16206,"['org-39940', 'org-37829', 'org-32434']"


In [67]:
currency_billing_df = combined_events_df[['id', 'currency', 'billing_cycle']]

In [68]:
currency_billing_df.head(5)

,id,currency,billing_cycle
0,acd43f22-d309-11e9-a770-00e04c6803e4,eur,monthly
1,acd43f23-d309-11e9-a770-00e04c6803e4,eur,yearly
2,acd43f24-d309-11e9-a770-00e04c6803e4,eur,monthly
3,acd43f25-d309-11e9-a770-00e04c6803e4,usd,yearly
4,acd43f26-d309-11e9-a770-00e04c6803e4,usd,yearly


In [69]:
events_acc_cur_bill_merge =  pd.merge(events_account_merge_df,
                                 currency_billing_df,
                                  on='id')

In [70]:
events_acc_cur_bill_merge.head(5)

,event_information_id,id,event_name,original_timestamp,recieved_at,url_event_was_triggered_on,account_sites_count,account_information_id,account_id,organization_id,currency,billing_cycle
0,1,acd43f22-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:11.918335,2019-08-27 14:17:15.918335,http://www.gdgt1cpt2ri0.com/w7dsvsbk/7n0ztux3/...,12,1,acc-13442,['org-49772'],eur,monthly
1,2,acd43f23-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:11.918335,2019-08-27 14:17:15.918335,http://www.qveak79goxia.com/8m6x73kjctp7m/yiiz...,13,2,acc-13025,['org-46185'],eur,yearly
2,3,acd43f24-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:14.918335,2019-08-27 14:17:15.918335,http://www.db4szx9qcg1aj.com/,17,3,acc-11689,"['org-48692', 'org-35895']",eur,monthly
3,4,acd43f25-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:13.918335,2019-08-27 14:17:15.918335,http://www.l2mdwl4jor0aid79.com/m8lcy9g2qnnk/1...,5,4,acc-16681,"['org-45640', 'org-31339', 'org-35178']",usd,yearly
4,5,acd43f26-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:12.918335,2019-08-27 14:17:15.918335,http://www.dxypuiklsc98e.com/58c9/p2061jvbu0/u...,13,5,acc-16206,"['org-39940', 'org-37829', 'org-32434']",usd,yearly


In [71]:
currency_dim_info = pd.read_sql('select * from currency_dim', con=conn)

In [72]:
currency_dim_info

,id,currency_name
0,1,eur
1,2,usd


In [73]:
billing_cycle_dim_info = pd.read_sql('select * from billing_cycle_dim', con=conn)

In [74]:
billing_cycle_dim_info

,id,billing_cycle
0,1,monthly
1,2,yearly


In [75]:
def fill_currency_id(row):
    if row['currency'] == 'eur':
        return 1
    else:
        return 2

In [76]:
def fill_billing_cycle_id(row):
    if row['billing_cycle'] == 'monthly':
        return 1
    else:
        return 2

In [77]:
events_acc_cur_bill_merge['currency_id'] = events_acc_cur_bill_merge.apply(fill_currency_id,
                                                                           axis=1)

In [78]:
events_acc_cur_bill_merge['billing_cycle_id'] = events_acc_cur_bill_merge.apply(fill_billing_cycle_id,
                                                                               axis=1)

In [79]:
events_acc_cur_bill_merge.head(5)

,event_information_id,id,event_name,original_timestamp,recieved_at,url_event_was_triggered_on,account_sites_count,account_information_id,account_id,organization_id,currency,billing_cycle,currency_id,billing_cycle_id
0,1,acd43f22-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:11.918335,2019-08-27 14:17:15.918335,http://www.gdgt1cpt2ri0.com/w7dsvsbk/7n0ztux3/...,12,1,acc-13442,['org-49772'],eur,monthly,1,1
1,2,acd43f23-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:11.918335,2019-08-27 14:17:15.918335,http://www.qveak79goxia.com/8m6x73kjctp7m/yiiz...,13,2,acc-13025,['org-46185'],eur,yearly,1,2
2,3,acd43f24-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:14.918335,2019-08-27 14:17:15.918335,http://www.db4szx9qcg1aj.com/,17,3,acc-11689,"['org-48692', 'org-35895']",eur,monthly,1,1
3,4,acd43f25-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:13.918335,2019-08-27 14:17:15.918335,http://www.l2mdwl4jor0aid79.com/m8lcy9g2qnnk/1...,5,4,acc-16681,"['org-45640', 'org-31339', 'org-35178']",usd,yearly,2,2
4,5,acd43f26-d309-11e9-a770-00e04c6803e4,subscription_created,2019-08-27 14:17:12.918335,2019-08-27 14:17:15.918335,http://www.dxypuiklsc98e.com/58c9/p2061jvbu0/u...,13,5,acc-16206,"['org-39940', 'org-37829', 'org-32434']",usd,yearly,2,2


In [80]:
event_info_fact_df = events_acc_cur_bill_merge[['account_sites_count',
                                               'currency_id',
                                               'billing_cycle_id',
                                               'event_information_id',
                                               'account_information_id']]

In [81]:
event_info_fact_df.head(5)

,account_sites_count,currency_id,billing_cycle_id,event_information_id,account_information_id
0,12,1,1,1,1
1,13,1,2,2,2
2,17,1,1,3,3
3,5,2,2,4,4
4,13,2,2,5,5


In [82]:
event_info_fact_df_records = event_info_fact_df.values.tolist()

In [83]:
event_info_fact_df_records[0:3]

[[12, 1, 1, 1, 1], [13, 1, 2, 2, 2], [17, 1, 1, 3, 3]]

### Inserting account dimension records into accounts_information_dim table

In [84]:
insert_records_in_table(event_info_fact_df_records, event_information_fact_table_insert)

Records inserted into the table
